## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-06-58-33 +0000,nyt,Gaza Hunger Presents Trump With Moral Test Fam...,https://www.nytimes.com/2025/07/30/us/politics...
1,2025-07-31-06-58-03 +0000,wapo,China summons Nvidia over backdoor security co...,https://www.washingtonpost.com/world/2025/07/3...
2,2025-07-31-06-52-04 +0000,bbc,Trump hits Brazil with 50% tariffs and sanctio...,https://www.bbc.com/news/articles/crlzn72eg25o...
3,2025-07-31-06-49-37 +0000,bbc,Airlines demand answers after air traffic cont...,https://www.bbc.com/news/articles/cd9jn1ydx9lo...
4,2025-07-31-06-33-00 +0000,wsj,British American Tobacco Backs Guidance on Bet...,https://www.wsj.com/business/earnings/british-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,57
19,tariffs,18
608,tsunami,18
98,trade,15
0,gaza,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
116,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...,129
282,2025-07-30-14-05-03 +0000,nypost,Trump threatens India with 25% tariff — plus ‘...,https://nypost.com/2025/07/30/us-news/trump-th...,125
248,2025-07-30-16-33-22 +0000,bbc,Trump to hit India with 25% tariffs - plus 'pe...,https://www.bbc.com/news/articles/cz71w1dgl84o...,122
105,2025-07-30-23-08-32 +0000,cbc,Trump slams Canada's plan to recognize Palesti...,https://www.cbc.ca/news/politics/us-canada-tra...,115
15,2025-07-31-05-42-58 +0000,nyt,Trump Threatens Canada Ahead of Trade Deadline...,https://www.nytimes.com/2025/07/31/us/politics...,108


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
116,129,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...
187,87,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
286,61,2025-07-30-13-51-11 +0000,cbc,Bank of Canada holds interest rate at 2.75% as...,https://www.cbc.ca/news/business/boc-july-deci...
304,59,2025-07-30-12-30-27 +0000,nyt,Hawaii Declares Emergency After Earthquake Off...,https://www.nytimes.com/2025/07/29/us/hawaii-t...
73,59,2025-07-31-00-53-35 +0000,nypost,NYC shooting victim Julia Hyman remembered as ...,https://nypost.com/2025/07/30/us-news/nyc-shoo...
120,47,2025-07-30-22-31-15 +0000,nypost,Witkoff could visit Gaza aid sites during Isra...,https://nypost.com/2025/07/30/world-news/witko...
198,42,2025-07-30-19-43-47 +0000,nypost,Texas Republicans pitch new House map to net t...,https://nypost.com/2025/07/30/us-news/texas-re...
18,41,2025-07-31-05-39-41 +0000,nyt,"Pro-Palestinian Group Can Appeal U.K. Ban, Jud...",https://www.nytimes.com/2025/07/30/world/europ...
282,41,2025-07-30-14-05-03 +0000,nypost,Trump threatens India with 25% tariff — plus ‘...,https://nypost.com/2025/07/30/us-news/trump-th...
212,40,2025-07-30-19-05-48 +0000,nypost,"Fed governors Michelle Bowman, Christopher Wal...",https://nypost.com/2025/07/30/business/fed-dec...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
